In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default="plotly_white"


In [4]:
data=pd.read_csv("retail_price.csv")
print(data.head())

  product_id product_category_name  month_year  qty  total_price  \
0       bed1        bed_bath_table  01-05-2017    1        45.95   
1       bed1        bed_bath_table  01-06-2017    3       137.85   
2       bed1        bed_bath_table  01-07-2017    6       275.70   
3       bed1        bed_bath_table  01-08-2017    4       183.80   
4       bed1        bed_bath_table  01-09-2017    2        91.90   

   freight_price  unit_price  product_name_lenght  product_description_lenght  \
0      15.100000       45.95                   39                         161   
1      12.933333       45.95                   39                         161   
2      14.840000       45.95                   39                         161   
3      14.287500       45.95                   39                         161   
4      15.100000       45.95                   39                         161   

   product_photos_qty  ...  comp_1  ps1        fp1      comp_2  ps2  \
0                   2  ...    89.

In [5]:
print((data.isnull()).sum())

product_id                    0
product_category_name         0
month_year                    0
qty                           0
total_price                   0
freight_price                 0
unit_price                    0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_score                 0
customers                     0
weekday                       0
weekend                       0
holiday                       0
month                         0
year                          0
s                             0
volume                        0
comp_1                        0
ps1                           0
fp1                           0
comp_2                        0
ps2                           0
fp2                           0
comp_3                        0
ps3                           0
fp3                           0
lag_price                     0
dtype: int64


In [6]:
print(data.describe())

              qty   total_price  freight_price  unit_price  \
count  676.000000    676.000000     676.000000  676.000000   
mean    14.495562   1422.708728      20.682270  106.496800   
std     15.443421   1700.123100      10.081817   76.182972   
min      1.000000     19.900000       0.000000   19.900000   
25%      4.000000    333.700000      14.761912   53.900000   
50%     10.000000    807.890000      17.518472   89.900000   
75%     18.000000   1887.322500      22.713558  129.990000   
max    122.000000  12095.000000      79.760000  364.000000   

       product_name_lenght  product_description_lenght  product_photos_qty  \
count           676.000000                  676.000000          676.000000   
mean             48.720414                  767.399408            1.994083   
std               9.420715                  655.205015            1.420473   
min              29.000000                  100.000000            1.000000   
25%              40.000000                  339.000

In [7]:
fig=px.histogram(data,
                 x="total_price",
                   nbins=20, 
                   title="Distribution of Total Price"
                 )
fig.show()

In [8]:
fig = px.box(data, 
             y='unit_price', 
             title='Box Plot of Unit Price')
fig.show()

In [ ]:
fig = px.scatter(data, 
                 x='qty', 
                 y='total_price', 
                 title='Quantity vs Total Price', trendline="ols")
fig.show()

In [11]:
fig = px.bar(data, x='product_category_name', 
             y='total_price', 
             title='Average Total Price by Product Category')
fig.show()

In [12]:
fig = px.box(data, x='weekday', 
             y='total_price', 
             title='Box Plot of Total Price by Weekday')
fig.show()

In [13]:
fig = px.box(data, x='holiday', 
             y='total_price', 
             title='Box Plot of Total Price by Holiday')
fig.show()

In [14]:
correlation_matrix = data.corr()
fig = go.Figure(go.Heatmap(x=correlation_matrix.columns, 
                           y=correlation_matrix.columns, 
                           z=correlation_matrix.values))
fig.update_layout(title='Correlation Heatmap of Numerical Features')
fig.show()

In [15]:
data["comp_price_diff"]=data["unit_price"]-data["comp_1"]
avg_price_diff_by_category=data.groupby("product_category_name")['comp_price_diff'].mean().reset_index()

fig = px.bar(avg_price_diff_by_category, 
             x='product_category_name', 
             y='comp_price_diff', 
             title='Average Competitor Price Difference by Product Category')
fig.update_layout(
    xaxis_title='Product Category',
    yaxis_title='Average Competitor Price Difference'
)
fig.show()

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
x=data[['qty','unit_price','comp_1','product_score','comp_price_diff']]
y=data["total_price"]
x_train,x_test,y_train,y_test=train_test_split(x,y,
                                               test_size=0.2,
                                               random_state=42
                                               )
model=DecisionTreeRegressor()
model.fit(x_train,y_train)

DecisionTreeRegressor()

In [17]:
y_pred = model.predict(x_test)
print(y_pred)
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode='markers', 
                         marker=dict(color='blue'), 
                         name='Predicted vs. Actual Retail Price'))
fig.add_trace(go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], 
                         mode='lines', 
                         marker=dict(color='red'), 
                         name='Ideal Prediction'))
fig.update_layout(
    title='Predicted vs. Actual Retail Price',
    xaxis_title='Actual Retail Price',
    yaxis_title='Predicted Retail Price'
)
fig.show()

[2680.   1680.    101.   2537.19  299.97  499.   3935.68 3711.7   648.89
  840.    249.5  2680.     78.    139.9   156.   8538.   2079.84  563.5
 5246.5    38.4  2336.1  2200.    299.5  1401.4  4769.7   379.6   179.99
  107.8   694.9  1737.4   248.   2088.7   179.1  2339.74 1399.86  140.
  419.79  227.96 2799.2   139.9   479.97  719.82  299.5  2468.6   699.8
  994.04  419.79   79.9   468.   2799.2   530.91  385.   3885.    479.8
  707.4   234.    419.4   548.9  1339.84  199.98  191.92 1019.94 1042.9
 1474.75   35.   1234.79 1601.06  339.96 3935.68  539.94 1421.67  943.8
  319.84 5280.    319.84  455.81  890.    495.5   239.6   388.7   468.
  973.   3499.    683.   4752.    595.    176.97  119.8   563.5   307.2
  569.9   179.7  1987.4   140.    149.7    79.9   648.89 4584.9   310.
  849.5   849.5   594.93   35.   1875.72  439.96  220.   6764.    770.
 3086.76  563.5  1188.   1559.1    29.5   781.3  1099.89  693.    299.4
 1559.1  1529.83  888.85  778.7   254.97  299.97 1188.6  1529.83 3